In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras as ks

pd.options.display.float_format = '{:.2f}'.format

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)
tf.keras.backend.set_floatx('float64')
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [18]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [19]:
df= pd.read_csv(
    "./resourses/contest_train.csv")



pd.options.display.float_format = '{:.2f}'.format
df.fillna(0, inplace=True)
df.set_index('ID', inplace=True) 



df.head()

,TARGET,FEATURE_0,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,FEATURE_8,...,FEATURE_250,FEATURE_251,FEATURE_252,FEATURE_253,FEATURE_254,FEATURE_255,FEATURE_256,FEATURE_257,FEATURE_258,FEATURE_259
ID,,,,,,,,,,,,,,,,,,,,,
1053371037,1,2.00,2.00,0.00,0.00,0.00,0.00,0.00,-114.53,-17.22,...,39.00,2.00,223.12,290.02,1.00,1.00,0.00,1.00,2.00,2.00
76507184,0,1.00,1.00,0.00,0.00,1.00,0.00,0.00,329.83,0.00,...,25.00,82.00,479.62,611.65,0.00,1.00,0.00,1.00,1.00,1.00
76753690,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,17.43,-6.68,...,119.00,0.00,352.27,564.57,1.00,1.00,0.00,1.00,1.00,1.00
1026167358,0,1.00,1.00,0.00,0.00,0.00,0.00,0.00,23.62,36.66,...,1.00,0.00,410.52,421.65,1.00,0.00,0.00,1.00,2.00,2.00
7220095746,0,4.00,3.00,1.00,0.00,1.00,0.00,0.00,677.75,0.00,...,21.00,0.00,439.17,476.19,1.00,1.00,0.00,1.00,1.00,1.00


In [20]:
dt = pd.read_csv(
    "./resourses/contest_test.csv")

dt_for = dt.copy()
dt.set_index('ID', inplace=True) 
dt.fillna(0, inplace=True) 
#dt.insert(0, 'TARGET', 1, True)

dt.head()

,FEATURE_0,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,FEATURE_8,FEATURE_9,...,FEATURE_250,FEATURE_251,FEATURE_252,FEATURE_253,FEATURE_254,FEATURE_255,FEATURE_256,FEATURE_257,FEATURE_258,FEATURE_259
ID,,,,,,,,,,,,,,,,,,,,,
1154270440,1.00,1.00,0.00,0.00,0.00,0.00,0.00,-46.95,-45.02,0.00,...,0.00,0.00,190.59,162.05,1.00,0.00,1.00,1.00,1.00,1.00
1147987574,0.00,0.00,1.00,0.00,0.00,0.00,0.00,343.63,0.00,3.00,...,5.00,109.00,594.57,685.50,1.00,1.00,0.00,1.00,1.00,2.00
1129622364,0.00,0.00,0.00,0.00,1.00,0.00,0.00,909.18,3260.45,3.00,...,0.00,297.00,10.21,300.40,0.00,0.00,0.00,1.00,1.00,1.00
619797496,6.00,2.00,0.00,0.00,0.00,0.00,0.00,-4.50,-93.37,0.00,...,1.00,2.00,334.87,363.74,1.00,1.00,1.00,2.00,2.00,1.00
7391484886,1.00,1.00,0.00,0.00,0.00,0.00,0.00,-23.19,-35.81,0.00,...,83.00,136.00,551.41,769.83,1.00,1.00,0.00,1.00,1.00,1.00


In [21]:
pd.options.display.float_format ='{:,.3f}'.format
dt_cop = dt.copy()
dt_train = np.array(dt_cop)
#dt_why = dt.pop('TARGET')

dt_train

array([[ 1.,  1.,  0., ...,  1.,  1.,  1.],
       [ 0.,  0.,  1., ...,  1.,  1.,  2.],
       [ 0.,  0.,  0., ...,  1.,  1.,  1.],
       ...,
       [52.,  1.,  0., ...,  1.,  1.,  1.],
       [ 1.,  1.,  0., ...,  1.,  1.,  1.],
       [25.,  1.,  0., ...,  1.,  2.,  1.]])

In [22]:
df_features = df.copy()
df_for = df.drop(columns=['TARGET'])
df_labels = pd.get_dummies(df['TARGET'])


In [23]:
df_features = np.array(df_features)
df_features

array([[1., 2., 2., ..., 1., 2., 2.],
       [0., 1., 1., ..., 1., 1., 1.],
       [1., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [1., 3., 2., ..., 1., 2., 1.],
       [0., 2., 2., ..., 1., 2., 1.]])

In [24]:
normalize = preprocessing.Normalization()

In [49]:
df_model = tf.keras.Sequential([
    layers.Dense(260),
    layers.Dense(130),
    layers.Dense(65),
    layers.Dense(3, activation='softmax')
])



df_model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
                      optimizer = tf.optimizers.Adam(),
                      metrics=[f1]
                )

In [50]:
df_model.fit(df_for, df_labels, epochs=250)

Epoch 1/250
767/767 [==============================] - 1s 2ms/step - loss: 97.4278 - f1: 0.5811
Epoch 2/250
767/767 [==============================] - 1s 2ms/step - loss: 14.7402 - f1: 0.5832
Epoch 3/250
767/767 [==============================] - 1s 2ms/step - loss: 11.6483 - f1: 0.5814
Epoch 4/250
767/767 [==============================] - 1s 2ms/step - loss: 8.5044 - f1: 0.5877
Epoch 5/250
767/767 [==============================] - 1s 2ms/step - loss: 9.2259 - f1: 0.5833
Epoch 6/250
767/767 [==============================] - 1s 2ms/step - loss: 6.0140 - f1: 0.5887
Epoch 7/250
767/767 [==============================] - 1s 2ms/step - loss: 6.6774 - f1: 0.5924
Epoch 8/250
767/767 [==============================] - 1s 2ms/step - loss: 4.0660 - f1: 0.5993
Epoch 9/250
767/767 [==============================] - 1s 2ms/step - loss: 3.7439 - f1: 0.6066
Epoch 10/250
767/767 [==============================] - 1s 2ms/step - loss: 3.7269 - f1: 0.6064
Epoch 11/250
767/767 [========================

767/767 [==============================] - 1s 2ms/step - loss: 0.6977 - f1: 0.7065
Epoch 87/250
767/767 [==============================] - 1s 2ms/step - loss: 0.6871 - f1: 0.7087
Epoch 88/250
767/767 [==============================] - 1s 2ms/step - loss: 0.6878 - f1: 0.7085
Epoch 89/250
767/767 [==============================] - 1s 2ms/step - loss: 0.6939 - f1: 0.7067
Epoch 90/250
767/767 [==============================] - 1s 2ms/step - loss: 0.6931 - f1: 0.7068
Epoch 91/250
767/767 [==============================] - 1s 2ms/step - loss: 0.7195 - f1: 0.7043
Epoch 92/250
767/767 [==============================] - 1s 2ms/step - loss: 0.6894 - f1: 0.7073
Epoch 93/250
767/767 [==============================] - 1s 2ms/step - loss: 0.7036 - f1: 0.7050
Epoch 94/250
767/767 [==============================] - 1s 2ms/step - loss: 0.7035 - f1: 0.7059
Epoch 95/250
767/767 [==============================] - 1s 2ms/step - loss: 0.7253 - f1: 0.6998
Epoch 96/250
767/767 [==============================]

767/767 [==============================] - 1s 2ms/step - loss: 0.6824 - f1: 0.7097
Epoch 171/250
767/767 [==============================] - 1s 2ms/step - loss: 0.6947 - f1: 0.7057
Epoch 172/250
767/767 [==============================] - 1s 2ms/step - loss: 0.7070 - f1: 0.7053
Epoch 173/250
767/767 [==============================] - 1s 2ms/step - loss: 0.7030 - f1: 0.7049
Epoch 174/250
767/767 [==============================] - 1s 2ms/step - loss: 0.6991 - f1: 0.7042
Epoch 175/250
767/767 [==============================] - 1s 2ms/step - loss: 0.6900 - f1: 0.7078
Epoch 176/250
767/767 [==============================] - 1s 2ms/step - loss: 12.7786 - f1: 0.6679
Epoch 177/250
767/767 [==============================] - 1s 2ms/step - loss: 0.7697 - f1: 0.6997
Epoch 178/250
767/767 [==============================] - 1s 2ms/step - loss: 0.7054 - f1: 0.7060
Epoch 179/250
767/767 [==============================] - 1s 2ms/step - loss: 0.6912 - f1: 0.7070
Epoch 180/250
767/767 [====================

In [156]:
prediction = df_model.predict(dt_train)

df_marks = pd.DataFrame(prediction)
mean = df_marks.nlargest(500, [1])

mean.insert(3, 'ID', dt_for['ID'], True)
del mean[0]
del mean[2]
mean.to_csv('contest_segmentss.csv', index = False)
print(mean)

         1          ID
4    1.000  7391484886
26   1.000  1148017175
30   1.000  3151488250
47   1.000   132847243
91   1.000   734771440
...    ...         ...
5294 1.000  1394129059
5313 1.000  4108915945
5315 1.000  6404399285
5318 1.000  3406963774
5320 1.000   377995630

[500 rows x 2 columns]


In [157]:
prediction = df_model.predict(dt_train)

df_marks = pd.DataFrame(prediction)
mean = df_marks.nlargest(200, [2])

mean.insert(3, 'ID', dt_for['ID'], True)
del mean[0]
del mean[1]
mean.to_csv('casesss.csv', index = False)
print(mean)

         2          ID
18   1.000  1433985624
36   1.000  1149159698
38   1.000   632762770
49   1.000  6397557852
63   1.000  6164987266
...    ...         ...
4786 1.000  3114271122
4796 1.000  1151090661
4799 1.000   568436200
4848 1.000   633599383
4892 1.000  3383949025

[200 rows x 2 columns]


In [159]:
print(np.argmax(prediction, axis=1))
dfabc = pd.DataFrame(np.argmax(prediction, axis=1))
dfabc.insert(1, 'ID', dt_for['ID'], True)
dfabc.to_csv('contest_answers.csv', index = False)

[1 0 0 ... 1 1 0]
